# Image Colorization
Dataset: [Realworld Occulated Faces(ROF)](https://github.com/ekremerakin/RealWorldOccludedFaces). Notebook adapted from Sreenivas Bhattiprolu, [Python for Microscopists](https://github.com/bnsreenu/python_for_microscopists)

In [ ]:
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

### Load images 

In [ ]:
source_path = "./Data/Color_Images/"

In [ ]:
#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train = tf.keras.utils.image_dataset_from_directory(
    source_path,
    image_size=(256, 256),
    batch_size=32,
    labels=None
)

In [ ]:
X =[]
Y =[]

for images in train:
    for img in images:
        try:
          lab = rgb2lab(img)
          X.append(lab[:,:,0]) 
          Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
          #so we divide the values by 128 to restrict values to between -1 and 1.
        except:
         print('error')
        
        
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

# Define Model

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

In [ ]:
#Decoder
#Decoder
#Note: For the last layer we use tanh instead of Relu. 
#This is because we are colorizing the image in this layer using 2 filters, A and B.
#A and B values range between -1 and 1 so tanh (or hyperbolic tangent) is used
#as it also has the range between -1 and 1. 
#Other functions go from 0 to 1.
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()

# Train Model

In [ ]:
model.fit(X,Y,validation_split=0.1, epochs=500, batch_size=32)
model.save('./Models/Colorize_Autoencoder500.h5')

# Evaluate Results 

In [ ]:
img1_color=[]

img1=img_to_array(load_img('./Data/Color_Images/Image1.png'))
img1 = resize(img1 ,(256,256))
img1_color.append(img1)

In [ ]:
img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))

In [ ]:
output1 = model.predict(img1_color)
output1 = output1*128


In [ ]:
result = np.zeros((256, 256, 3))
result[:,:,0] = img1_color[0][:,:,0]
result[:,:,1:] = output1[0]

In [ ]:
out_img=lab2rgb(result)

In [ ]:
out_img

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(load_img('./Data/Color_Images/Image1.png'))
plt.show()

# Assuming out_img is your NumPy array
plt.imshow(out_img)
plt.show()